In [9]:
# Comment faire une boucle pour avoir les données sur chacune de mes villes ? 
# où concerver les données ? 

# Prévisions météorologiques à 16 jours de 77 stations balnéaires Françaises 

Étape 1, scraping de la liste des stations balnéaires Françaises

In [1]:
# Import des librairies 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import json_normalize

In [2]:
# Scapping des noms de stations balnéaires 
website_urls = ['https://www.france-voyage.com/tourisme/stationsbalneaires-panorama_1.htm','https://www.france-voyage.com/tourisme/stationsbalneaires-panorama_2.htm','https://www.france-voyage.com/tourisme/stationsbalneaires-panorama_3.htm','https://www.france-voyage.com/tourisme/stationsbalneaires-panorama_4.htm']
cite_balneaires = []
for urls in website_urls:
    html = requests.get(urls).content
    soup = BeautifulSoup(html, "html")
    cite_table = soup.find_all('div',{'class':'title nolabels'})
    noms = [nom_ville.text.strip() for nom_ville in cite_table]
    cite_balneaires.append(noms)
cite_balneaires = [s for page in cite_balneaires for s in page]

In [3]:
# Scrapping des noms des départements
departements = []
for urls in website_urls:
    html = requests.get(urls).content
    soup = BeautifulSoup(html, "html")
    departements_table = soup.find_all('div',{'class':'depinfo'})
    noms_dep_1 = [dep.text.strip() for dep in departements_table]
    noms_dep = [dep[:-11] for dep in noms_dep_1]
    departements.append(noms_dep)

departements = [d for pages in departements for d in pages]

In [4]:
# Créer un dictionnaire avec ville / dept
ville_departement = dict(zip(cite_balneaires ,departements))

In [5]:
# Transformer en liste d'éléments uniques 
ville_dept_items = ville_departement.items()
ville_dept_list = list(ville_dept_items)
df_ville_dept = pd.DataFrame(ville_dept_list, columns=['city','departements'])
stations_balneaire = (list(df_ville_dept.city.unique()))

In [6]:
df_ville_dept

,city,departements
0,Ajaccio,Corse-du-Sud
1,Andernos-les-Bains,Gironde
2,Anglet,Pyrénées-Atlantiques
3,Arcachon,Gironde
4,Argelès-sur-Mer,Pyrénées-Orientales
...,...,...
72,Saint-Valery-sur-Somme,Somme
73,Sainte-Anne,Martinique
74,Soulac-sur-Mer,Gironde
75,Vieux-Boucau Port d'Albret,Landes


In [20]:
# Noms des sations en majuscule (pour requête API)
stations_balneaire = [x.upper() for x in stations_balneaire]

Accéder à l'API Weatherbit

In [30]:
# Import API key
import_key = open('./ignore.txt','r').read()
import_key

'8f948672fb704039a9f6e6c0a453e761'

In [31]:
# accès à l'API WEATHER FORECAST 
# https://api.weatherbit.io/v2.0/forecast/daily?city=Raleigh,NC&key=API_KEY
api_access = requests.get(f'https://api.weatherbit.io/v2.0/forecast/daily?city=AJACCIO&country=FR&key={import_key}')

In [23]:
cols = ['sunset_ts',
'wind_gust_spd',
'wind_spd',
'wind_dir',
'clouds',
'sunrise_ts',
'pop',
'uv',
'precip',
'low_temp',
'min_temp',
'max_temp',
'high_temp',
'temp',
'datetime',
'weather.description']

In [32]:
api_access.status_code

429

In [26]:
longitude = api_access.json()['lon']
latitude = api_access.json()['lat']
weather_data = json_normalize(api_access.json()['data'])
weather_table = pd.DataFrame(weather_data)[cols]
# Faire après : .set_index('datetime')

KeyError: 'lon'

In [ ]:
# remplacer le nom de la ville par station 
weather_table['city'] = 'Ajaccio'
weather_table['long'] = longitude
weather_table['lat'] = latitude
weather_table

In [12]:
list(df_ville_dept.city)

['Ajaccio',
 'Andernos-les-Bains',
 'Anglet',
 'Arcachon',
 'Argelès-sur-Mer',
 'Banyuls-sur-Mer',
 'Barneville-Carteret',
 'Biarritz',
 'Bidart',
 'Biscarrosse-Plage',
 'Bray-Dunes',
 'Cabourg',
 'Cancale et ses huîtres',
 'Canet-en-Roussillon',
 'Capbreton',
 'Carcans-Plage',
 'Châtelaillon-Plage',
 'Ciboure',
 'Deauville',
 'Dieppe',
 'Dinard',
 'Étretat',
 'Fort-Mahon-Plage',
 'Fréjus',
 'Granville',
 'Gruissan',
 'Hendaye',
 'Juan-les-Pins',
 "L'Île-Rousse",
 "L'Isle-Adam",
 'La Baule',
 'La Grande-Motte',
 'La pointe de Grave',
 "La pointe du Bout et l'anse Mitan",
 "La presqu'île de Quiberon",
 'La Trinité-sur-Mer',
 'Lacanau',
 "Le Cap d'Agde",
 'Le Croisic',
 'Le Diamant',
 'Le golfe de Porto',
 'Le Gosier',
 'Le Grau-du-Roi',
 'Le Touquet-Paris-Plage',
 'Les falaises de Meschers-sur-Gironde',
 "Les Sables-d'Olonne",
 'Les Saintes-Maries-de-la-Mer',
 'Leucate',
 'Mers-les-Bains',
 'Mimizan-Plage',
 'Narbonne-Plage',
 'Palavas-les-Flots',
 'Piriac-sur-Mer',
 'Pornic',
 'Port-Ba

In [17]:
api_access = requests.get(f'https://api.weatherbit.io/v2.0/forecast/daily?city=BARNEVILLE-CARTERET&country=FR&key={import_key}')

In [21]:
# boucle for pour api request
cols = ['sunset_ts',
'wind_gust_spd',
'wind_spd',
'wind_dir',
'clouds',
'sunrise_ts',
'pop',
'uv',
'precip',
'low_temp',
'min_temp',
'max_temp',
'high_temp',
'temp',
'datetime',
'weather.description']


df_list = []
    
for station in stations_balneaire :
    try:
        api_access = requests.get(f'https://api.weatherbit.io/v2.0/forecast/daily?city={station}&country=FR&key={import_key}')
        longitude = api_access.json()['lon']
        latitude = api_access.json()['lat']
        weather_data = json_normalize(api_access.json()['data'])
        weather_table = pd.DataFrame(weather_data)[cols]
        weather_table['city'] = station
        weather_table['long'] = longitude
        weather_table['lat'] = latitude
        df_list.append(weather_table)
        print(station)
    except:
        print('Skipping station', station)

Skipping station AJACCIO
Skipping station ANDERNOS-LES-BAINS
Skipping station ANGLET
Skipping station ARCACHON
Skipping station ARGELÈS-SUR-MER
Skipping station BANYULS-SUR-MER
Skipping station BARNEVILLE-CARTERET
Skipping station BIARRITZ
Skipping station BIDART
Skipping station BISCARROSSE-PLAGE
Skipping station BRAY-DUNES
Skipping station CABOURG
Skipping station CANCALE ET SES HUÎTRES
Skipping station CANET-EN-ROUSSILLON
Skipping station CAPBRETON
Skipping station CARCANS-PLAGE
Skipping station CHÂTELAILLON-PLAGE
Skipping station CIBOURE
Skipping station DEAUVILLE
Skipping station DIEPPE
Skipping station DINARD
Skipping station ÉTRETAT
Skipping station FORT-MAHON-PLAGE
Skipping station FRÉJUS
Skipping station GRANVILLE
Skipping station GRUISSAN
Skipping station HENDAYE
Skipping station JUAN-LES-PINS
Skipping station L'ÎLE-ROUSSE
Skipping station L'ISLE-ADAM
Skipping station LA BAULE
Skipping station LA GRANDE-MOTTE
Skipping station LA POINTE DE GRAVE
Skipping station LA POINTE DU B

In [266]:
final_weather_df = pd.concat(df_list)
final_weather_df

ValueError: No objects to concatenate

API AIR QUALITY

In [254]:
# boucle for pour api request air quality
cols_air_quality = ['aqi', 'datetime']

df_air_quality_list = []
    
for station in stations_balneaire :
    try:
        api_access_air_quality = requests.get(f'https://api.weatherbit.io/v2.0/history/airquality?city={station}&country=FR&key={key}')
        city = api_access_air_quality.json()['city_name']
        air_quality_data = json_normalize(api_access_air_quality.json()['data'])
        air_quality_table = pd.DataFrame(air_quality_data)[cols]
        air_quality_table['city'] = station
        df_air_quality_list.append(air_quality_table)
        #print(station)
    except:
        print('Skipping station', station)

Skipping station Ajaccio
Skipping station Andernos-les-Bains
Skipping station Anglet
Skipping station Arcachon
Skipping station Argelès-sur-Mer
Skipping station Banyuls-sur-Mer
Skipping station Barneville-Carteret
Skipping station Biarritz
Skipping station Bidart
Skipping station Biscarrosse-Plage
Skipping station Bray-Dunes
Skipping station Cabourg
Skipping station Cancale et ses huîtres
Skipping station Canet-en-Roussillon
Skipping station Capbreton
Skipping station Carcans-Plage
Skipping station Châtelaillon-Plage
Skipping station Ciboure
Skipping station Deauville
Skipping station Dieppe
Skipping station Dinard
Skipping station Étretat
Skipping station Fort-Mahon-Plage
Skipping station Fréjus
Skipping station Granville
Skipping station Gruissan
Skipping station Hendaye
Skipping station Juan-les-Pins
Skipping station L'Île-Rousse
Skipping station L'Isle-Adam
Skipping station La Baule
Skipping station La Grande-Motte
Skipping station La pointe de Grave
Skipping station La pointe du B

In [247]:
final_air_quality_df = pd.concat(df_air_quality_list)
final_air_quality_df

,aqi,datetime,city
0,50.6,2020-08-01:13,Ajaccio
1,50.2,2020-08-01:12,Ajaccio
2,49.2,2020-08-01:11,Ajaccio
3,47.8,2020-08-01:10,Ajaccio
4,42.0,2020-08-01:09,Ajaccio
...,...,...,...
67,50.0,2020-07-29:18,Saint-Gilles-Croix-de-Vie
68,52.0,2020-07-29:17,Saint-Gilles-Croix-de-Vie
69,53.0,2020-07-29:16,Saint-Gilles-Croix-de-Vie
70,52.0,2020-07-29:15,Saint-Gilles-Croix-de-Vie


In [248]:
final_air_quality_df['datetime'].str.extract(r'(\d\d\d\d-\d\d-\d\d):(\d\d)')

,0,1
0,2020-08-01,13
1,2020-08-01,12
2,2020-08-01,11
3,2020-08-01,10
4,2020-08-01,09
...,...,...
67,2020-07-29,18
68,2020-07-29,17
69,2020-07-29,16
70,2020-07-29,15


In [249]:
# Split time date & Hours 
final_air_quality_df = pd.concat([final_air_quality_df, final_air_quality_df['datetime'].str.extract(r'(\d\d\d\d-\d\d-\d\d):(\d\d)')],axis=1)


In [253]:
# Average par jour
final_air_quality_df = final_air_quality_df.groupby([0,'city'])['aqi'].mean()

KeyError: 'Column not found: aqi'

In [251]:
# Traitement noms des colonnes
final_air_quality_df = (final_air_quality_df.drop('datetime',axis=1))
final_air_quality_df = (final_air_quality_df.drop(1,axis=1))

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [252]:
final_air_quality_df = final_air_quality_df.rename({0: 'datetime'}, axis=1)
final_air_quality_df

0           city                     
2020-07-29  Ajaccio                      42.100000
            Andernos-les-Bains           51.100000
            Anglet                       43.000000
            Arcachon                     51.100000
            Argelès-sur-Mer              50.000000
                                           ...    
2020-08-01  Saint-Cyprien                65.585714
            Saint-Florent                55.100000
            Saint-François               70.100000
            Saint-Gilles-Croix-de-Vie    26.200000
            Étretat                      20.642857
Name: aqi, Length: 192, dtype: float64

In [228]:
stations_baleneaires_database = pd.concat([final_air_quality_df], [final_weather_df], [df_ville_dept]).groupby(['city'], axis=1)
#final_air_quality_df.groupby([0,'city'])['aqi'].mean()

ValueError: Only can inner (intersect) or outer (union) join the other axis

In [ ]:
# average aqi sur la journée
# merge avec les clés datetime et city 

# Enregister en csv
# dataframe_name.to_csv(path="infos du path", index=false)
# to_sql
# to_excel